In [24]:
import cv2
import numpy as np
import re
from tqdm import tqdm
import os

In [25]:
#read train img and masks
#stride s=128, window size w=512 (w x w)
#calculate ratio r for all cropped img
#compare min(r) with theta
#keep crop img if min(r) > theta, theta = 0.01
#double check from massachusetts 48,290 img in total

In [26]:
class cvpr_converter:
    def __init__(self, root_dir,save_dir):        
        self.root_dir = root_dir
        self.save_dir = save_dir
        mask_files = [f for f in os.listdir(os.path.join(root_dir, 'mask'))] # algo: List Comprehension
        img_files = [f for f in os.listdir(os.path.join(root_dir, 'image'))]
        #self.ids = [t[t.index('tile_') + 5: t.index('.')] for t in img_files]
        self.ids = [t[0:t.index('_sat')] for t in img_files]        
        
    def convertAllToInput(self):
        self.convertToInput(0, len(self.ids))
        
    def convertToInput(self, start, end):    
        def sliding_window (x,w_width,y,w_height,image):
            window = image[x:x + w_width, y:y + w_height, :]
            return window

        def save_img (img_id,x,y,window,image=bool):           
            path_img_s = self.save_dir+ 'image' + ''
            path_mask_s = self.save_dir+ 'mask' + ''
            if image == True:
                cv2.imwrite(os.path.join(path_img_s,str(img_id)+'img'+str(x)+ '_' + str(y)+'.tiff'),window)
            else:
                cv2.imwrite(os.path.join(path_mask_s,str(img_id)+'mask'+str(x)+ '_' + str(y)+'.tif'),window)

        def balanced_with_road (window2,theta):
            n_white_road = np.sum(window2 == 255)
            n_black_bg = np.sum(window2 == 0)
            n_total_pix = n_white_road+n_black_bg #n=3,000,000 rgb bands

            if n_white_road + n_black_bg != n_total_pix:
                print('Pixel number of black and white does not add up to total number of pixel.')
            else:
                r = [n_white_road/n_total_pix, n_black_bg/n_total_pix]
                if min(r) > theta:
                    return True
                else: 
                    return False

        def slice_select_save (image,mask,img_id):
            stepSize = 128
            (w_width, w_height) = (512, 512) # window size
            theta = 0.01
            size = 1024

            for x in range(0, size - w_width +1 , stepSize):
                for y in range(0, size - w_height +1, stepSize):
                    window = sliding_window (x,w_width,y,w_height,image)        
                    window2 = sliding_window (x,w_width,y,w_height,mask)
                    if balanced_with_road (window2,theta) == True:
                        save_img(img_id,x,y,window,True)
                        save_img(img_id,x,y,window2,False)
                        
        pbar = tqdm(self.ids)
        
        for img_id in pbar:
            path_fol = self.root_dir+ 'image' + ''
            path_img = path_fol + '/' + img_id + "_sat.jpg"
            path_fol2 = self.root_dir+ 'mask' + ''
            path_mask = path_fol2 + '/' + img_id + "_mask.png"
            
            image = cv2.imread(path_img) 
            mask = cv2.imread(path_mask)
                    
            slice_select_save (image,mask,img_id)
            

      

In [27]:
#sliding window algorithm (medium level on leetcode) ?

In [ ]:
#read into variable
root_dir = '/mnt/data2/Projects/RoadDetection/1_SIINet-master/dataset_cvpr/'
save_dir = '/mnt/data2/Projects/RoadDetection/1_SIINet-master/processed_dataset/'

converter = cvpr_converter(root_dir,save_dir)
converter.convertAllToInput()

 51%|█████▏    | 3200/6226 [20:50<56:21,  1.12s/it]  